In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import make_pipeline

from sktime.transformations.panel.rocket import Rocket

import pandas as pd

In [2]:
path = "/Users/nraman/Documents/thesis_videos/"

lower = [76, 77, 90, 180, 85, 16, 315, 404, 320, 307]

upper = [184, 74, 73, 72, 11, 302, 303, 304, 408, 306]

u2 = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409]
l2 = [291, 375, 321, 405, 314, 17, 84, 181, 91, 146]

u3 = [57, 186, 92, 165, 167, 164, 393, 391, 322, 410]
l3 = [287, 273, 335, 406, 313, 18, 83, 182, 106, 43]

combo_indices = lower + upper + u2 + l2 + u3 + l3

In [3]:
subfolders = [i for i in os.listdir(path) if ".mp4" not in i and ".DS_Store" not in i]

def find_pointfolder(subfolder, path = "/Users/nraman/Documents/thesis_videos/"):
    cur_path = f"{path}{subfolder}"
    # get names of files within the subfolder
    subfiles = os.listdir(f"{path}{subfolder}")
    nps = [i for i in subfiles if ".npy" in i]
    
    if(len(nps) > 1):
        return("Error: Multiple np arrays saved in the folder")
    else:
        array = np.load(f"{cur_path}/{nps[0]}")
        array = array[:, combo_indices]
        return(array.reshape(array.shape[0], len(combo_indices)*2))

In [4]:
ground_folders = [i for i in subfolders if "ground" in i]
speak_folders = [i for i in subfolders if "ground" not in i]

ground_data = []
speak_data = []

for g in ground_folders:
    ground_data.append(find_pointfolder(g))

ground_data = np.concatenate(ground_data)

for s in speak_folders:
    speak_data.append(find_pointfolder(s))

In [5]:
#pd.DataFrame(eval([speak_data[0][:, i] for i in range(len(combo_indices))]))
trainsize = 70
trainplus = 40
window = 12

df=pd.DataFrame(np.zeros([trainsize, len(combo_indices)*2]))
df=df.astype(object)  # this line allows the signment of the array

for j in range(trainsize):
    for a in range(len(combo_indices)*2):
        if(j < trainplus):
            df.iloc[j, a] = pd.Series(speak_data[j][:, a][:window])
        else:
            df.iloc[j, a] = pd.Series(ground_data[(j-trainplus)*window: ((j-trainplus)+1)*window][:, a])

In [6]:
testsize = 20
testplus = 7
buffer = 12*(trainsize - trainplus)

testX=pd.DataFrame(np.zeros((testsize, len(combo_indices)*2))).astype(object)

for j in range(testsize):
    for a in range(len(combo_indices)*2):
        if(j < testplus):
            testX.iloc[j, a] = pd.Series(speak_data[-j][:, a][:12])
        else:
            testX.iloc[j, a] = pd.Series(ground_data[(j-testplus)*12 + buffer: ((j-testplus)+1)*12 + buffer][:, a])

In [7]:
ytrain = np.concatenate([np.ones(trainplus), np.zeros(trainsize - trainplus)])
ytest = np.concatenate([np.ones(testplus), np.zeros(testsize - testplus)])

In [8]:
# df is for training
#train_y = np.concatenate([np.ones(30), np.zeros(30)])

rocket = Rocket()  # by default, ROCKET uses 10,000 kernels
rocket.fit(df)
X_train_transform = rocket.transform(df)

In [9]:
classifier = RidgeClassifier(solver = "sag", random_state = 50)
classifier.fit(X_train_transform, ytrain)

RidgeClassifier(random_state=50, solver='sag')

In [10]:
classifier.score(rocket.transform(testX), ytest)

0.95